## Observation Maker

1.  Enter source data in csv file to simulate patient generated wt observations

 - Hard Code: 

Patient use existing Synthea patient: Eldon718 Halvorson124  Patient/06e1f0dd-5fbe-4480-9bb4-6b54ec02d31b

 - Variables: 

    - measurement
    - datetime
    - context ? ( see OMH )
    - device info
    - sensed or entered
    - SR Key?
    
- Add Reliability Tags 
- Add example name and description
- Validate
- Save

In [1]:
%cd '/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker'

/Users/ehaas/Documents/Python/MyNotebooks/Obs_maker


In [3]:
from fhir.resources.observation import Observation
from fhir.resources import construct_fhir_element

none_list = [None,{}]
tag = True # add reliability tags to Observation 
#tag = False # no reliability tags to Observation 
# usually done on the server side see notes below...

###  Import from YAML file Observation template

for now fixed:

- patient
- code
- category
- status
- valueQuantiry units, code

In [11]:
obs_obj = Observation.parse_file("fhir_templates/Observation_a1c.yml")#"Observation_wt.yml")
#print(obs_obj.yaml())
print(obs_obj.category[0].coding)

[Coding(resource_type='Coding', fhir_comments=None, extension=None, id=None, id__ext=None, code='laboratory', code__ext=None, display='Laboratory', display__ext=None, system='http://terminology.hl7.org/CodeSystem/observation-category', system__ext=None, userSelected=None, userSelected__ext=None, version=None, version__ext=None)]


### Functions to create fhir type dicts from data

scrubs null from data too

return only if object not empty else return None

datatypes covered:

- Identifier
- Reference
- Coding
- Extension

In [5]:
def new_dict(**kwargs):
    none_list = [None,{}]
    new_dict = {}
    for k,v in kwargs.items():
        #print(k,v)
        new_dict[k] = v
        #print(new_dict)
    new_dict = {k: v for k, v in new_dict.items() if v==v and v not in none_list} # scrub dict of None   
    if new_dict:
        return new_dict

### function to add complex elements to resource

In [6]:
def update_obs(element,element_dict, resource = obs_obj):
    element_type = next(i.type_.__resource_type__ for i in resource.element_properties() if i.name == element)
    #print(f'element={element}\nelement_dict={element_dict}\nelement_type={element_type}')
    if element_dict:
        new_element=construct_fhir_element(element_type,element_dict)
    else:
        return
    try: # assume not a list 
        setattr(resource,element,new_element) # not a list 
        #print(getattr(resource,element).yaml())
    except: # treat as list
        try: # assume list already exists
            getattr(resource,element).append(new_element)
        except: # list does not exist
            setattr(resource,element,[new_element])
        #print(getattr(resource,element)[0].yaml())

### update the effective dataTime

In [7]:
obs_obj.effectiveDateTime = row.effective_datetime

NameError: name 'row' is not defined

### update the encounter

what code or codes should be used for this  ( "at home measuremet")?

todo ... review patent empowerment work.

In [ ]:
element = 'encounter'
element_dict = new_dict(
            reference = row.encounter_fhirid,
            identifier = new_dict(value = row.encounter_identifier),  # business identifier
            display = row.encounter_display,
            )
update_obs(element,element_dict)

### update the value

 for now assume in pounds and only update the decimal value

In [ ]:

obs_obj.valueQuantity.value = row.value
#print(obs_obj.valueQuantity.yaml())

### Add Device info which provides data provenance

options

- fhir id
- identifier  ( can this be overloaded to indicate the identifier for device without doing a reference ? )
- string 

In [ ]:
element = 'device'
element_dict = new_dict(
            reference = row.device_fhirid,
            identifier = new_dict(value = row.device_identifier),  # business identifier
            display = row.device_display,
            )
update_obs(element,element_dict)

### Add SR Key to `basedOn` element

options 

- fhir id
- identifier ( can this be overloaded to indicate the identifier for device without doing a reference ? )
- string


In [ ]:
element = 'basedOn'
element_dict = new_dict(
            reference = row.servicerequest_fhirid,
            identifier = new_dict(value = row.servicerequest_identifier),  # business identifier
            display = row.servicerequest_display,
            )
update_obs(element,element_dict)

### Add any device/gateway/service created identifier to observation

see OMH for examples

In [ ]:
element ='identifier'
element_dict = new_dict(
                value = row.identifier,
                assigner = new_dict(display=row.assigner,)
                )
update_obs(element,element_dict)


### Add source to observation

[meta.source](http://build.fhir.org/resource-definitions.html#Meta.source) is one way to provide a simple provenance using for example, a business identifier

this one option for providing provenance in addition to or instead of Device, basedOn, Identifier, Encounter or a full Provenance resource.

In [ ]:
if row.source == row.source:
    obs_obj.meta.source = row.source

### Add modality extension observation

uri: `http://www.fhir.org/guides/omhtofhir/StructureDefinition/extension-modality`

see [OMH](https://healthedata1.github.io/mFHIR/StructureDefinition-extension-modality.html)

*NOTE: This extension was originally defined to go on the device element, but think is better on the Obervation*

|Code|Display|Definition|
|---|---|---|
|sensed|Sensed|Device measurement is sensed directly by the device|
|self-reported|Self Reported|Device measurement is entered by the user|



In [ ]:
element = 'extension'
if row.modality==row.modality:
    element_dict = new_dict(
                    valueCode = row.modality,
                    url = 'http://www.fhir.org/guides/omhtofhir/StructureDefinition/extension-modality',
                    )
    update_obs(element,element_dict)

### Add reliability tags to observation

Using Josh's suggested codes as a starting point...

>All data coming via the patient are submitted with a tag like {"system": "http://terminology.hl7.org/CodeSystem/common-tags", "code": "patient-supplied"}
FHIR searches find resources with this tag by default; clients that don't want them can filter them out, through search params
Providers and EHRs can do what they want with these data (e.g., leaving them tagged; reviewing and adding a provider-reviewed tag; reviewing and stripping the patient-supplied tag; deleting them...)

for now need own system:

url = `http://www.fhir.org/guides/argonaut/argo-write/CodeSystem/tags`

|Code|Display|Definition|
|---|---|---|
|patient-supplied|Patient Supplied Data|Data is supplied by patient - either patient generated data or data generated elsewhere and forwarded by patient (todo get references to definitions of PGD)|
|provider-reviewed|Provider Reviewed Data|Data is supplied by patient and has been reviewed by provider ( either manully or through some automated fashion)|

In [ ]:
if tag:
    url = 'http://www.fhir.org/guides/argonaut/argo-write/CodeSystem/tags'
    tag = "patient-supplied"
    element = 'tag'
    element_dict = new_dict(system = 'http://www.fhir.org/guides/argonaut/argo-write/CodeSystem/tags',
                        code = tag,
                                   )

    update_obs(element,element_dict,resource = obs_obj.meta)

### Add description and example name extension to observation

this annotates each example for future documentation

In [ ]:
element = 'extension'
if row.instance_name==row.instance_name:
    element_dict = new_dict(
                    valueString = row.instance_name,
                    url = 'http://hl7.org/fhir/StructureDefinition/instance-name',
                    )
    update_obs(element,element_dict,resource = obs_obj.meta)

In [ ]:
element = 'extension'
if row.instance_description==row.instance_description:
    element_dict = dict(
                    valueMarkdown = row.instance_description,
                    url = 'http://hl7.org/fhir/StructureDefinition/instance-description',
                    )
    update_obs(element,element_dict,resource = obs_obj.meta)

In [ ]:
print(obs_obj.yaml())

### Validate

In [ ]:
from requests import post
from IPython.display import display as Display, HTML
def validate(data):

    #fhir_test_server = 'http://test.fhir.org/r4'
    #fhir_test_server = 'http://hapi.fhir.org/baseR4'
    fhir_test_server = 'http://wildfhir4.aegis.net/fhir4-0-1'
    
    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    r = post(f'{fhir_test_server}/Observation/$validate', headers = headers, data = data)
    return r

print('...validating')
r = validate(obs_obj.json())
r.status_code, r.json()
Display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))

### Save Locally

In [ ]:
from pathlib import Path
ig_source_path = 'examples'
if tag:
    path = Path.cwd() / ig_source_path / f'{obs_obj.meta.extension[0].valueString}_tagged.yaml'
else:
    path = Path.cwd() / ig_source_path / f'{obs_obj.meta.extension[0].valueString}.yaml'   
print(f'...........saving to file {path}............')
path.write_text(obs_obj.yaml())